In [1]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])
if manual:
    %matplotlib inline

In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none func

In [2]:
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.messaging as message
from biogeme import expressions as ex
import pandas as pd
import numpy as np
import xlsxwriter

# Calibration
## Estimation of the model parameters
quetzal_germany is being estimated using [PandasBiogeme](https://biogeme.epfl.ch/). This notebook estimates calibration parameters for the model's utility functions.
- Documentation and reference: [Bierlaire, M. (2020). A short introduction to PandasBiogeme. Technical report TRANSP-OR 200605. Transport and Mobility Laboratory, ENAC, EPFL.](https://transp-or.epfl.ch/documents/technicalReports/Bier20.pdf)
- Tutorial: https://www.youtube.com/watch?v=OiM94B8WayA

### Model formulation
The model consists of systematic utility functions, one for each mode. They comprise an alternaive-specific constant (ASC), a distance-dependent part with travel time and cost summarised as generalised cost (GC), and a cost damping function F

$V_i = ASC + F(GC(T, C), b_{gc_i})$

Index i marks the demand group. I = {'commuting' (1), 'education' (2), 'shopping/medical' (3), 'business' (4), 'private' (6)}

Note: The cost variable already includes subscriptions

In [3]:
input_path = '../input/'
model_path = '../model/'

### Prepare the database

In [4]:
df = pd.read_csv(input_path + 'transport_demand/calibration_inter-cellular_trips_MiD2017.csv')
df = df[['cost_rail_short', 'cost_rail_long', 'cost_car', 'cost_coach', 'cost_bus', 'cost_walk', 'cost_air',
         'time_rail_short', 'time_rail_long', 'time_car', 'time_coach', 'time_bus', 'time_walk', 'time_air',
         'cost_rail', 'cost_road', 'time_rail', 'time_road',
         'mode_model', 'purpose_vp', 'car_avail', 'length']]
df.columns = ['C_RAIL_S', 'C_RAIL_L', 'C_CAR', 'C_COACH', 'C_BUS', 'C_NON_MOTOR', 'C_AIR',
              'T_RAIL_S', 'T_RAIL_L', 'T_CAR', 'T_COACH', 'T_BUS', 'T_NON_MOTOR', 'T_AIR',
              'C_RAIL', 'C_ROAD', 'T_RAIL', 'T_ROAD',
              'MODE', 'PURPOSE', 'CAR_AV', 'DIST']

In [5]:
inf = 1000
df = df.replace({np.inf:inf})

In [6]:
df.describe()

,C_RAIL_S,C_RAIL_L,C_CAR,C_COACH,C_BUS,C_NON_MOTOR,C_AIR,T_RAIL_S,T_RAIL_L,T_CAR,...,T_NON_MOTOR,T_AIR,C_RAIL,C_ROAD,T_RAIL,T_ROAD,MODE,PURPOSE,CAR_AV,DIST
count,94452.000000,94452.000000,94452.000000,94452.000000,94452.000000,94452.0,94452.000000,94452.000000,94452.000000,94452.000000,...,94452.000000,94452.000000,94452.000000,94452.000000,94452.000000,94452.000000,94452.000000,94452.000000,94452.000000,94452.000000
mean,14.749359,85.570365,8.286941,7.534644,3.253165,0.0,963.556645,98.825857,586.793029,79.797397,...,453.336250,974.322868,17.074625,4.373309,84.787540,195.806821,5.552863,3.651622,0.965676,94.665749
std,12.905439,56.321519,11.890400,6.687144,1.268850,0.0,175.408933,127.813356,435.822611,84.476973,...,449.761943,125.048510,16.929394,3.730686,90.915057,306.849972,1.349977,2.059639,0.186202,134.622888
min,0.000000,19.000000,0.010944,5.000000,0.000000,0.0,44.162224,12.433333,16.033333,10.400000,...,6.683333,4.697222,0.000000,0.000000,12.433333,13.200000,1.000000,1.000000,0.000000,0.265000
25%,6.202913,20.494609,2.342865,5.000000,2.740000,0.0,1000.000000,30.283333,115.483333,36.350000,...,76.633333,1000.000000,7.359172,3.600000,30.283333,40.533333,6.000000,1.000000,1.000000,29.479000
50%,10.906830,139.000000,4.454365,5.000000,4.000000,0.0,1000.000000,45.900000,1000.000000,49.683333,...,120.316667,1000.000000,12.205805,4.000000,45.066667,68.300000,6.000000,3.000000,1.000000,44.912000
75%,17.689693,139.000000,8.405009,5.000000,4.000000,0.0,1000.000000,98.852083,1000.000000,74.600000,...,1000.000000,1000.000000,19.229493,4.000000,95.200000,145.804167,6.000000,6.000000,1.000000,79.174500
max,42.000000,139.000000,119.532004,59.142925,4.000000,0.0,1000.000000,1000.000000,1000.000000,682.283333,...,1000.000000,1000.000000,139.000000,48.193655,1000.000000,1000.000000,7.000000,6.000000,9.000000,1044.928000


In [7]:
# Scale time to hours
df['T_RAIL_Sc'] = df['T_RAIL'] / 60
df['T_RAIL_S_S'] = df['T_RAIL_S'] / 60
df['T_RAIL_L_S'] = df['T_RAIL_L'] / 60
df['T_CAR_S'] = df['T_CAR'] / 60
df['T_COACH_S'] = df['T_COACH'] / 60
df['T_BUS_S'] = df['T_BUS'] / 60
df['T_ROAD_S'] = df['T_ROAD'] / 60
df['T_AIR_S'] = df['T_AIR'] / 60
df['T_NON_MOTOR_S'] = df['T_NON_MOTOR'] / 60

In [8]:
# Make car availability binary
df['CAR_AV'] = df['CAR_AV'].replace({9:0})

In [9]:
# Add PT availabilities
df['RAIL_AV'] = (df['T_RAIL']!=inf).astype(int)
df['RAIL_S_AV'] = (df['T_RAIL_S']!=inf).astype(int)
df['RAIL_L_AV'] = (df['T_RAIL_L']!=inf).astype(int)
df['COACH_AV'] = (df['T_COACH']!=inf).astype(int)
df['BUS_AV'] = (df['T_BUS']!=inf).astype(int)
df['ROAD_AV'] = (df['T_ROAD']!=inf).astype(int)
df['AIR_AV'] = (df['T_AIR']!=inf).astype(int)
df['NON_MOTOR_AV'] = (df['C_NON_MOTOR']!=inf).astype(int)

In [10]:
# Merge rail and road PT
df['MODE'] = df['MODE'].replace({2:1, 3:4})
len(df.loc[df['MODE'].isin([2,3])])

0

In [11]:
# Remove trips where mode is car but the car availability is zero
# because it irritates the MLE algorithm
mask = ((df['MODE']==6) & (df['CAR_AV']==0))
drop_ratio = len(df.loc[mask])/len(df.loc[df['MODE']==6])
lengths = list(df.loc[mask, 'DIST'])
print('Share of car trips dropped: {}'.format(drop_ratio))
df = df.loc[~mask]
# Drop other modes equally
for m in [1,4]:
    n_drops = int(np.round(drop_ratio * len(df.loc[df['MODE']==m])))
    df = df.drop(df.loc[(df['MODE']==m) & (df['DIST']<max(lengths)) &
                        (df['DIST']>min(lengths))].sample(n_drops).index)
    print('Dropped ' + str(n_drops) + ' trips of mode ' + str(m))
print(len(df))

Share of car trips dropped: 0.014148505211025496
Dropped 106 trips of mode 1
Dropped 45 trips of mode 4
93124


In [12]:
# Check that PT trips have no infinity cost
for mode, col in zip([1, 4, 5], ['RAIL_AV', 'ROAD_AV', 'AIR_AV']):
    assert len(df.loc[((df['MODE']==mode) & ~(df[col]))].index) == 0, str(mode)

### Model variables
All columns are variables. DefineVariable creates a new column in the database.

#### Cost damping

Many modelling studies have shown that cost damping is required in order to flatten the tail of time/cost elasticities, i.e. decrease the impact of long distances on choice results to prevent from overestimation of time/cost parameters. Cost damping represents the property of decreasing marginal utility. It is commonly approached with Box-Cox transformations of generalised cost (usually defined as the sum of travel time and travel expenditures divided by the value of time). Daly (2010) proposes a hybrid function as a sum of the linear term and the narural logarithm of the same. However, the linear term still dominates cost on long distances. Rich (2015), main developer of the Danish National Transport Model, proposes a more complex spline function which successfully manages cost damping and even outperforms the Box-Cox transformation in terms of stability of elasticities.

In [13]:
# Using the hybrid linear-logarithmic version requires a new column
# for the logarithm of time in minutes
'''df['T_RAIL_S'] = np.log(df['T_RAIL'])
df['T_CAR_S'] = np.log(df['T_CAR'])
df['T_COACH_S'] = np.log(df['T_COACH'])
df['T_BUS_S'] = np.log(df['T_BUS'])
df['T_AIR_S'] = np.log(df['T_AIR'])
df['T_NON_MOTOR_S'] = np.log(df['T_NON_MOTOR'])'''
print('This functional form performes poorly')

This functional form performes poorly


#### Generalised cost

Both, travel time and monetary cost should be affected by cost damping measures. It is logical to define a generalised cost term `GC` with dimension of time units. This requires definition or estimation of values of time, in order to rescale monetary units, for all segments. Usually, the value of time (VoT) is distance-dependent. In the case of Germany, VoT can be taken from research undertaken within the Federal Government's transport study "Bundesverkehrswegeplan 2030": Axhausen et al. 2015. Ermittlung von Bewertungsansätzen für Reisezeiten und Zuverlässigkeit auf der Basis eines Modells für modale Verlagerungen im nicht-gewerblichen und gewerblichen Personenverkehr für die Bundesverkehrswegeplanung

In [14]:
# VoT from literature, distance-dependent, see cal19
VoT = pd.read_csv(input_path + 'vot.csv', header=[0,1], index_col=0)
VoT.sample(2)

,root,Fz1,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,...,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,Fz4,Fz6
,all,all,all,all,all,all,PT,PT,PT,PT,...,air,air,air,air,car,car,car,car,car,car
775,21.6875,20.7275,9.18,11.9,29.27,16.2325,16.2925,12.385,13.48,16.815,...,81.3975,81.3975,210.2025,45.395,21.4275,20.3025,12.6875,27.0225,23.8575,16.2325
550,18.6050,17.8250,9.18,11.9,23.15,14.1850,14.0650,10.900,13.48,15.330,...,55.6350,55.6350,111.1350,32.030,18.4350,17.5350,11.1350,23.4450,20.1450,14.1850


In [15]:
# Make distance integer
df['DIST'] = df['DIST'].apply(int)

In [16]:
df = df.loc[df['DIST']<=1000]
len(df)

93121

In [17]:
# Generate generalised cost
VoT = VoT.to_dict()
df['GC_RAIL'] = df['T_RAIL_Sc'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                   for c,d,p in zip(df['C_RAIL'], df['DIST'], df['PURPOSE'])]
df['GC_RAIL_S'] = df['T_RAIL_S_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                      for c,d,p in zip(df['C_RAIL_S'], df['DIST'], df['PURPOSE'])]
df['GC_RAIL_L'] = df['T_RAIL_L_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                      for c,d,p in zip(df['C_RAIL_L'], df['DIST'], df['PURPOSE'])]
df['GC_COACH'] = df['T_COACH_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                    for c,d,p in zip(df['C_COACH'], df['DIST'], df['PURPOSE'])]
df['GC_BUS'] = df['T_BUS_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                for c,d,p in zip(df['C_BUS'], df['DIST'], df['PURPOSE'])]
df['GC_ROAD'] = df['T_ROAD_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                  for c,d,p in zip(df['C_ROAD'], df['DIST'], df['PURPOSE'])]
df['GC_AIR'] = df['T_AIR_S'] + [c / VoT[('Fz'+str(p), 'air')][d]
                                for c,d,p in zip(df['C_AIR'], df['DIST'], df['PURPOSE'])]
df['GC_CAR'] = df['T_CAR_S'] + [c / VoT[('Fz'+str(p), 'car')][d]
                                for c,d,p in zip(df['C_CAR'], df['DIST'], df['PURPOSE'])]
df['GC_NON_MOTOR'] = df['T_NON_MOTOR_S']

In [18]:
# Create the initial database and make columns global variables
database = db.Database('MiD2017', df.copy())
globals().update(database.variables)
database.getSampleSize()

93121

### Estimation parameters

In [19]:
asc_rail = ex.Beta('asc_rail', 0, None, None, 0)
asc_rail_s = ex.Beta('asc_rail_s', 0, None, None, 0)
asc_rail_l = ex.Beta('asc_rail_l', 0, None, None, 0)
asc_coach = ex.Beta('asc_coach', 0, None, None, 0)
asc_bus = ex.Beta('asc_bus', 0, None, None, 0)
asc_road = ex.Beta('asc_road', 0, None, None, 0)
asc_air = ex.Beta('asc_air', 0, None, None, 0)
asc_car = ex.Beta('asc_car', 0, None, None, 1)
asc_non_motor = ex.Beta('asc_non_motor', 0, None, None, 0)

In [20]:
b_t = ex.Beta('b_t', 0, None, None, 0)
b_c = ex.Beta('b_c', 0, None, None, 0)
b_ac = ex.Beta('b_ac', 0, None, None, 0)

In [21]:
# non-linear time component
b_t2 = ex.Beta('b_t2', 0, None, None, 0)

In [22]:
# generalised cost function parameter
b_gc = ex.Beta('b_gc', 0, None, None, 0)

In [23]:
# Parameters for the nested logit structure
mu_pt = ex.Beta('mu_pt', 1, 1, 10, 0)

### Utility functions

In [24]:
'''# Aggregated formulation
V_RAIL = asc_rail + b_t * T_RAIL_S + b_c * C_RAIL + b_ac * AC_RAIL
V_COACH = asc_coach + b_t * T_COACH_S + b_c * C_COACH + b_ac * AC_COACH
V_BUS = asc_bus + b_t * T_BUS_S + b_c * C_BUS + b_ac * AC_BUS
V_AIR = asc_air + b_t * T_AIR_S + b_c * C_AIR + b_ac * AC_AIR
V_CAR = asc_car + b_t * T_CAR_S + b_c * C_CAR
V_NON_MOTOR = asc_non_motor + b_t * T_NON_MOTOR_S

# Aggregated formulation with non-linear perception of travel time
V_RAIL = asc_rail + b_t * T_RAIL + b_t2 * T_RAIL_S + b_c * C_RAIL + b_ac * AC_RAIL
V_COACH = asc_coach + b_t * T_COACH + b_t2 * T_COACH_S + b_c * C_COACH + b_ac * AC_COACH
V_BUS = asc_bus + b_t * T_BUS + b_t2 * T_BUS_S + b_c * C_BUS + b_ac * AC_BUS
V_AIR = asc_air + b_t * T_AIR + b_t2 * T_AIR_S + b_c * C_AIR + b_ac * AC_AIR
V_CAR = asc_car + b_t * T_CAR + b_t2 * T_CAR_S + b_c * C_CAR
V_NON_MOTOR = asc_non_motor + b_t * T_NON_MOTOR + b_t2 * T_NON_MOTOR_S'''
print('Deprected')

Deprected


In [25]:
# The cost damping function
def spline(x, beta=b_gc, Q=3, c0=0, c1=20, c2=40, c3=np.inf):
    alpha = [0, -beta/2*ex.Power(ex.log(c1),3),
             -beta/2*ex.log(c1)*(3*ex.Power(ex.log(c2),2)+ex.Power(ex.log(c1),2))] # for Q=3
    theta = [1, 3/2*ex.log(c1), 3*ex.log(c1)*ex.log(c2)] # for Q=3
    def component(x, q):
        return beta*theta[q-1]*ex.Power(ex.log(x),Q-q+1) + alpha[q-1]
    if x < c1:
        return component(x,1)
    elif x < c2:
        return component(x,2)
    elif x >= c2:
        return component(x,3)

In [26]:
# Aggregated formulation with damped generalised cost
V_RAIL = asc_rail + spline(GC_RAIL)
V_RAIL_S = asc_rail_s + spline(GC_RAIL_S)# + b_ac * AC_RAIL
V_RAIL_L = asc_rail_l + spline(GC_RAIL_L)# + b_ac * AC_RAIL
V_COACH = asc_coach + spline(GC_COACH)# + b_ac * AC_COACH
V_BUS = asc_bus + spline(GC_BUS)# + b_ac * AC_BUS
V_ROAD = asc_road + spline(GC_ROAD)
V_AIR = asc_air + spline(GC_AIR)# + b_ac * AC_AIR
V_CAR = asc_car + spline(GC_CAR)
V_NON_MOTOR = asc_non_motor + spline(GC_NON_MOTOR)

### Run the estimation

In [27]:
# Define level of verbosity
logger = message.bioMessage()
#logger.setSilent()
logger.setWarning()
#logger.setGeneral()
#logger.setDetailed()

In [28]:
# Map modes to utility functions
V = {1:V_RAIL,
#     1:V_RAIL_S,
#     2:V_RAIL_L,
#     3:V_COACH,
     4:V_ROAD,#BUS,
     5:V_AIR,
     6:V_CAR,
     7:V_NON_MOTOR}

In [29]:
# Map the availability of alternatives with MODE as key
# PT is always available
av = {1:RAIL_AV,#RAIL_S_AV,
#      2:RAIL_L_AV,
#      3:COACH_AV,
      4:ROAD_AV,#BUS_AV,
      5:AIR_AV,
      6:CAR_AV,
      7:NON_MOTOR_AV}

In [30]:
# Mode nests as tuples with nest name and dictionary where
# alternative IDs are mapped to alpha values. Missing ID's alpha is zero
# Alternatively use lists with mode ID without alpha
pt = mu_pt, [1, 4]#[1, 2, 3, 4]
air = 1, [5]
car = 1, [6]
walk = 1, [7]
nests = pt, air, car, walk

In [31]:
# Write results to an Excel file
writer = pd.ExcelWriter(input_path + 'estimation_results_car_own.xls', engine='xlsxwriter')

In [32]:
# Choose the multinomial logit model
mnl = models.loglogit(V, av, MODE)

In [33]:
model_mnl = bio.BIOGEME(database, mnl)
model_mnl.modelName = 'MNL'

In [34]:
results_mnl = model_mnl.estimate()

In [35]:
results = results_mnl.getEstimatedParameters()
for key, val in results_mnl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-5.130389,0.174772,-29.3547,0,0.209027,-24.5442,0
asc_non_motor,-4.577224,0.0420303,-108.903,0,0.042261,-108.309,0
asc_rail,-2.300821,0.0145939,-157.657,0,0.0197695,-116.382,0
asc_road,-3.247691,0.0190027,-170.907,0,0.0196273,-165.468,0
b_gc,-0.293137,0.00820167,-35.7411,0,0.0201964,-14.5143,0
Number of estimated parameters,5.000000,,,,,,
Sample size,93121.000000,,,,,,
Excluded observations,0.000000,,,,,,
Init log likelihood,-126285.393650,.7g,,,,,
Final log likelihood,-36886.390068,.7g,,,,,


In [36]:
results.to_excel(writer, sheet_name=model_mnl.modelName)

In [37]:
# Choose the logarithmic nested logit model
nl = models.lognested(V, av, nests, MODE)

In [38]:
# Nested Logit
model_nl = bio.BIOGEME(database, nl)
model_nl.modelName = 'NL'

In [39]:
results_nl = model_nl.estimate()

In [40]:
results = results_nl.getEstimatedParameters()
for key, val in results_nl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Active bound,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-5.129075,0,0.202561,-25.3211,0,0.488719,-10.4949,0
asc_non_motor,-4.576706,0,0.0420424,-108.859,0,0.0426343,-107.348,0
asc_rail,-2.300823,0,0.0285625,-80.554,0,0.112875,-20.3838,0
asc_road,-3.247678,0,0.129443,-25.0896,0,0.575833,-5.63996,1.70087e-08
b_gc,-0.293137,0,0.0228205,-12.8453,0,0.0956106,-3.06595,0.00216983
mu_pt,1.000000,1,0.11902,8.40195,0,0.535139,1.86867,0.0616681
Number of estimated parameters,6.000000,,,,,,,
Sample size,93121.000000,,,,,,,
Excluded observations,0.000000,,,,,,,
Init log likelihood,-126285.393650,.7g,,,,,,


In [41]:
results.to_excel(writer, sheet_name=model_nl.modelName)

In [42]:
# Run the estimation by purpose
results = []
for p in [1,2,3,4,6]:
    for car_ownership in [0,1]:
        database = db.Database('MiD2017', df.copy())
        database.remove(PURPOSE!=p)
        database.remove(CAR_AV!=car_ownership)
        print('Sample size for purpose {}, car_ownership {}: {}'.format(
            p, car_ownership, database.getSampleSize()))
        model = bio.BIOGEME(database, mnl) # Choose the model formulation
        model.modelName = 'MNL_Fz' + str(p) + '_' + str(car_ownership) # Name it
        results.append(model.estimate()) # Estimation
        output = results[-1].getEstimatedParameters()
        # Add results to the Excel file
        for key, val in results[-1].getGeneralStatistics().items():
            output.loc[key] = [val[0], val[1]] + ['' for i in range(len(output.columns)-2)]
        output.to_excel(writer, sheet_name=model.modelName)

Sample size for purpose 1, car_ownership 0: 629
Sample size for purpose 1, car_ownership 1: 24565
Sample size for purpose 2, car_ownership 0: 115
Sample size for purpose 2, car_ownership 1: 3400
Sample size for purpose 3, car_ownership 0: 386
Sample size for purpose 3, car_ownership 1: 22489
Sample size for purpose 4, car_ownership 0: 94
Sample size for purpose 4, car_ownership 1: 5499
Sample size for purpose 6, car_ownership 0: 826
Sample size for purpose 6, car_ownership 1: 35118


In [43]:
writer.save()